In [4]:
'''if accessing google colab'''
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
import keras
import keras.backend as K
import tensorflow as tf
import numpy as np
import cv2
  
from __future__ import division
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, UpSampling2D, Reshape, core, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K
from keras.utils.vis_utils import plot_model as plot
from keras.optimizers import SGD
from keras.optimizers import *
from keras.layers import *  
import matplotlib.pyplot as plt
try:
    from keras_efficientnets import EfficientNetB0 
except ModuleNotFoundError:
    print('installing keras_efficientnets...')
    !pip install keras_efficientnets
    from keras_efficientnets import EfficientNetB0
def get_result(file):
    model_weights = '/content/gdrive/My Drive/final_ct results/finel.hdf5'
    segmentation_model_weights = '/content/gdrive/My Drive/final_ct results/weight_lung.hdf5'
    def model(weights = model_weights):
        net = EfficientNetB0(
        include_top=False,
        input_shape=(512 ,512 ,3) ,
        weights = 'imagenet' ,
        dropout_rate = 0.4)
        x = keras.layers.Input((512 ,512 ,3))
        x_ = keras.layers.Input((512 ,512 ,1))
        multiply = keras.layers.Multiply()([x ,x_])
        #pre = keras.layers.Lambda(preprocess)(multiply)
        final = net(multiply)
        conv1 = keras.layers.Conv2D(512 ,kernel_size = (2 ,2) ,strides = 2 ,kernel_initializer='glorot_uniform' ,activation = 'elu')(final)
        drop1 = keras.layers.Dropout(0.4)(conv1)
        conv2 = keras.layers.Conv2D(512 ,kernel_size = (3 ,3) ,kernel_initializer='glorot_uniform' ,activation = 'elu' ,name = 'pen_ultimate')(drop1)   
        pool1 = keras.layers.MaxPooling2D((6 ,6))(conv2)
        flatten1 = keras.layers.Flatten()(pool1)
        drop2 = keras.layers.Dropout(0.4)(flatten1)
        dense1 = keras.layers.Dense(512 ,activation = 'elu')(drop2)
        dense1 = keras.layers.Dense(1 ,activation = 'sigmoid' ,name = 'final')(dense1)
        classifier = keras.models.Model(inputs = [x ,x_],outputs = dense1)
        classifier.load_weights(weights)
        return classifier
    def BCDU_net_D3(input_size = (256,256,1)):
        N = input_size[0]
        inputs = Input(input_size) 
        conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
        conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
  
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
        conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
        conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
        conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
        conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
        drop3 = Dropout(0.5)(conv3)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
        # D1
        conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)     
        conv4_1 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
        drop4_1 = Dropout(0.5)(conv4_1)
    # D2
        conv4_2 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(drop4_1)     
        conv4_2 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4_2)
        conv4_2 = Dropout(0.5)(conv4_2)
    # D3
        merge_dense = concatenate([conv4_2,drop4_1], axis = 3)
        conv4_3 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge_dense)     
        conv4_3 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4_3)
        drop4_3 = Dropout(0.5)(conv4_3)

        up6 = Conv2DTranspose(256, kernel_size=2, strides=2, padding='same',kernel_initializer = 'he_normal')(drop4_3)
        up6 = BatchNormalization(axis=3)(up6)
        up6 = Activation('relu')(up6)

        x1 = Reshape(target_shape=(1, np.int32(N/4), np.int32(N/4), 256))(drop3)
        x2 = Reshape(target_shape=(1, np.int32(N/4), np.int32(N/4), 256))(up6)
        merge6  = concatenate([x1,x2], axis = 1) 
        merge6 = ConvLSTM2D(filters = 128, kernel_size=(3, 3), padding='same', return_sequences = False, go_backwards = True,kernel_initializer = 'he_normal' )(merge6)
            
        conv6 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
        conv6 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

        up7 = Conv2DTranspose(128, kernel_size=2, strides=2, padding='same',kernel_initializer = 'he_normal')(conv6)
        up7 = BatchNormalization(axis=3)(up7)
        up7 = Activation('relu')(up7)

        x1 = Reshape(target_shape=(1, np.int32(N/2), np.int32(N/2), 128))(conv2)
        x2 = Reshape(target_shape=(1, np.int32(N/2), np.int32(N/2), 128))(up7)
        merge7  = concatenate([x1,x2], axis = 1) 
        merge7 = ConvLSTM2D(filters = 64, kernel_size=(3, 3), padding='same', return_sequences = False, go_backwards = True,kernel_initializer = 'he_normal' )(merge7)
        
        conv7 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
        conv7 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

        up8 = Conv2DTranspose(64, kernel_size=2, strides=2, padding='same',kernel_initializer = 'he_normal')(conv7)
        up8 = BatchNormalization(axis=3)(up8)
        up8 = Activation('relu')(up8)    

        x1 = Reshape(target_shape=(1, N, N, 64))(conv1)
        x2 = Reshape(target_shape=(1, N, N, 64))(up8)
        merge8  = concatenate([x1,x2], axis = 1) 
        merge8 = ConvLSTM2D(filters = 32, kernel_size=(3, 3), padding='same', return_sequences = False, go_backwards = True,kernel_initializer = 'he_normal' )(merge8)    
    
        conv8 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
        conv8 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
        conv8 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
        conv9 = Conv2D(1, 1, activation = 'sigmoid')(conv8)

        model = Model(inputs = inputs, output = conv9)
        model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])    
        return model
    '''Loading segmentation model along with weights'''
    seg = BCDU_net_D3(input_size = (512,512,1))
    seg.load_weights(segmentation_model_weights)
    '''loading the classifier with pretrained weights'''
    model = model()
    '''Reading the given file and resizing it'''
    img_gray = cv2.imread(file ,cv2.IMREAD_GRAYSCALE)
    img_gray = cv2.resize(img_gray ,(512 ,512))
    img = cv2.imread(file)
    img = cv2.resize(img ,(512 ,512))
    print('predicting mask....')
    mask = seg.predict(np.expand_dims(img_gray ,axis = (0 ,3)))
    print('predicting class...')
    result = model.predict([np.expand_dims(img ,axis = 0) ,mask])
    if result >= 0.5:
        print('result : covid')
    else:
        print('result : non-covid')
file = input('Enter the filepath:')
get_result(file)